![logo](./img/TheBridge_RL.png)

# Taxi Autónomo (Smartcab), CON RL
### *Segunda Parte: Evaluación*

## Contenidos

* [Inicializacion y setup](#Inicializacion-y-setup)  

* [Evaluacion](#Evaluacion)  

* [Comparando nuestro agente de Q-learning con no usar Aprendizaje por Refuerzo](#Comparando-nuestro-agente-de-Q-learning-con-no-usar-Aprendizaje-por-Refuerzo)  



### Inicializacion y setup
  

[al indice](#Contenidos)  

Necesarios para volver al punto donde lo dejamos en la primera parte, ejecuta las celdas hasta llegar a la cabecera: "Evaluación"

In [ ]:
import gym
import warnings


env = gym.make("Taxi-v3", render_mode = "ansi").env

In [ ]:
env.reset(seed = 19)
print(env.render())
print("Current State:", env.s)
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

In [ ]:
movements = [2,0]
for mov in movements:
    env.step(mov)
    print(env.render())
    print("State:",env.s)

In [ ]:
from time import sleep
from IPython.display import clear_output

def episode_animation(frames):
    for i, frame in enumerate(frames): # Recorremos todo el conjunto de frames
        clear_output(wait=True) # Limpiamos la "pantalla"
        print(frame['frame']) # Visualizamos el "pantallazo" resultado de cada acción
        print(f"Timestep: {i + 1}") # Aumentamos el contador de pasos/steps
        # Imprimimos el resto de valores correspondientes a cada frame y que hemos guardado al realizar el "aprendizaje"
        print(f"State: {frame['state']}") 
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Elapsed time (sec.): {frame['elapsed']}")
        sleep(.1) # "Dormimos" el programa un tiempo para que nuestro ojo pueda ver la imagen antes de borrarla y mostrar la siguiente

In [ ]:
import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [ ]:
alpha = 0.05
gamma = 0.9
epislon = 0.1

In [ ]:

import random

all_epochs = []
all_penalties = []
num_episodes = 100000 
state = env.s

for i in range(1, num_episodes +1): 
    epochs, penalties, reward = 0,0,0
    done = False
    
    while not done: 
        if random.uniform(0,1) < epislon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        next_state, reward, done, truncated, info = env.step(action)
        old_value = q_table[state,action]
        next_max = np.max(q_table[next_state]) # maxQ(s',a')
        
        new_value = (1 - alpha)*old_value + alpha * (reward + gamma * next_max)
        
        q_table[state,action] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait = True)
        print(f"Episode: {i}, {i/num_episodes * 100:.2f}%")
    state,info = env.reset()
    
print("Entrenamiento finalizado")
    

### Evaluacion  

[al indice](#Contenidos)  



Vamos a evaluar el rendimiento de nuestro agente. Ya no necesitamos explorar acciones, así que ahora la siguiente acción siempre se selecciona usando el mejor valor Q:


In [ ]:
# NO EJECUTES EL CODIGO DIRECTAMENTE PORQUE SE ENCARGA DE CALCULAR Q
# HAY QUE MODIFICARLO PRIMERO PARA QUE REALMENTE USE LA TABLA Q

all_epochs = []
all_penalties = []
num_episodes = 100000 
state = env.s

for i in range(1, num_episodes +1): 
    epochs, penalties, reward = 0,0,0
    done = False
    
    while not done: 
        if random.uniform(0,1) < epislon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        next_state, reward, done, truncated, info = env.step(action)
        old_value = q_table[state,action]
        next_max = np.max(q_table[next_state]) # maxQ(s',a')
        
        new_value = (1 - alpha)*old_value + alpha * (reward + gamma * next_max)
        
        q_table[state,action] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait = True)
        print(f"Episode: {i}, {i/num_episodes * 100:.2f}%")
    state,info = env.reset()

Los valores parecen bastante buenos, 12.5 pasos de duración media y no hay penalizaciones (no se intenta recoger o dejar al pasajero en sitios equivocados)

Utilicemos ahora la visualización para ver cuanto de bien ha aprendido a conducir. Vamos a analizar 5 episodios escogidos aleatoriamente.

Bastante bien, ¿no? Comparemos ahora con el "entrenamiento", por llamarlo de alguna forma, sin aprendizaje por refuerzo

### Comparando nuestro agente de Q-learning con no usar Aprendizaje por Refuerzo
  

[al indice](#Contenidos)  



Vamos a evaluar a nuestros agentes de acuerdo con las siguientes métricas,

* Número promedio de penalizaciones por episodio: Cuanto menor sea el número, mejor será el rendimiento de nuestro agente. Idealmente, nos gustaría que esta métrica sea cero o muy cercana a cero.
* Número promedio de pasos por episodio: También queremos que sea un valor pequeño, que nuestro agente tome la ruta más corta para llegar al destino.
* Recompensas promedio por movimiento: Una recompensa más grande significa que el agente está haciendo lo correcto. Es por eso que decidir las recompensas es una parte crucial del Aprendizaje por Refuerzo.


Recuperemos el código que ya desarrollamos en la sesión sin aprendizaje por refuerzo para obtener los valores anteriores para este escenario y hacer la comparativa

In [ ]:
"""Evaluate agent's performance without Q-learning"""

total_epochs, total_penalties, total_rewards = 0, 0, 0
episodes = 100

for _ in range(episodes):
    env.reset()
    # Crea el estado inicial
    state = env.encode(3, 1, 2, 0)
    env.s = state
    # Inicializa las epochs, penalties y rewards
    epochs, penalties, reward = 0, 0, 0

    done = False
    actions = []
    while not done:
        # Elige la acción random
        action = env.action_space.sample()
        actions.append(action)
        # Ejecuta la accion
        state, reward, done, truncated, info = env.step(action)
        total_rewards += reward
        # Actualiza el valor de penalties si el reward es -10
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per step: {total_rewards/total_epochs}")

Rellena la tabla y observa la comparación

| **Medida**              | **Rendimiento Agente Aleatorio** | **Rendimiento Agente Q-Learning** |
|---------------------------|-----------------------------|-------------------------------|
| Average penalties per episode                   |                  ||
| Average timesteps per episode                   |              |                     |
| Average reward per action|        |                  |

Estas métricas se calcularon a lo largo de 100 episodios. ¡Y como muestran los resultados, nuestro agente de Q-learning lo clavó!
